In [12]:
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
import os
import json
import time
import pickle
import requests

conn = psycopg2.connect( 
                        host = 'project.cgxhdwn5zb5t.us-east-1.rds.amazonaws.com',
                        port = 5432, 
                        user = 'postgres',
                        password = 'Admin123',
                        database = 'postgres')
cursor = conn.cursor()

In [13]:
DEC2FLOAT = psycopg2.extensions.new_type(
    psycopg2.extensions.DECIMAL.values,
    'DEC2FLOAT',
    lambda value, curs: float(value) if value is not None else None)
psycopg2.extensions.register_type(DEC2FLOAT)

In [14]:
cursor.execute('SELECT * FROM household_181')
rows = cursor.fetchall()
col_names = []
for elt in cursor.description:
    col_names.append(elt[0])

household_181 = pd.DataFrame(data=rows, columns=col_names )

In [15]:
household_181.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136245 entries, 0 to 136244
Columns: 182 entries, control to year
dtypes: float64(181), object(1)
memory usage: 189.2+ MB


In [16]:
household_181.dtypes

control      float64
omb13cbsa    float64
division     float64
intstatus    float64
intmonth     float64
              ...   
rmhome       float64
rmcosts      float64
rmhood       float64
rmother      float64
year          object
Length: 182, dtype: object

In [ ]:
household_181.describe()

In [5]:
#find the index of target column
household_181.columns.get_loc("ratingnh")

155

In [6]:
#drop the target column
df = household_181.drop(columns=['ratingnh'])

In [7]:
#total column should be 180 since we drop target column
df.describe()

,control,omb13cbsa,division,intstatus,intmonth,intlang,intmode,permnear,vacresdays,vacrntdays,...,movforce,rmjob,rmownhh,rmfamily,rmchange,rmcommute,rmhome,rmcosts,rmhood,rmother
count,1.362450e+05,136245.000000,136245.000000,136245.000000,136245.000000,136245.000000,136245.000000,136245.000000,136245.000000,136245.000000,...,136245.000000,136245.000000,136245.000000,136245.000000,136245.000000,136245.000000,136245.000000,136245.000000,136245.000000,136245.000000
mean,1.104325e+07,62040.862857,5.478014,1.245829,7.039422,0.586686,1.165973,-5.759257,-5.748776,-5.775280,...,-4.383104,-4.514646,-4.547917,-4.529216,-4.515535,-4.509597,-4.554347,-4.524151,-4.551088,-4.524247
std,2.483225e+04,35455.723682,2.556581,0.645094,1.409371,2.328762,2.370249,1.421780,1.559736,1.386489,...,3.260303,3.139458,3.076451,3.112814,3.139171,3.150524,3.064042,3.124800,3.074021,3.123453
min,1.100000e+07,12060.000000,1.000000,1.000000,4.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,...,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000
25%,1.102171e+07,31080.000000,3.000000,1.000000,6.000000,1.000000,1.000000,-6.000000,-6.000000,-6.000000,...,-6.000000,-6.000000,-6.000000,-6.000000,-6.000000,-6.000000,-6.000000,-6.000000,-6.000000,-6.000000
50%,1.104339e+07,42660.000000,5.000000,1.000000,7.000000,1.000000,2.000000,-6.000000,-6.000000,-6.000000,...,-6.000000,-6.000000,-6.000000,-6.000000,-6.000000,-6.000000,-6.000000,-6.000000,-6.000000,-6.000000
75%,1.106477e+07,99998.000000,8.000000,1.000000,8.000000,1.000000,2.000000,-6.000000,-6.000000,-6.000000,...,-6.000000,-6.000000,-6.000000,-6.000000,-6.000000,-6.000000,-6.000000,-6.000000,-6.000000,-6.000000
max,1.108703e+07,99999.000000,9.000000,3.000000,10.000000,10.000000,2.000000,2.000000,3.000000,3.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000


In [8]:
from sklearn.preprocessing import LabelEncoder

# Extract our X and y data
#X = df.iloc[:,0:50]
X = df
y = household_181["ratingnh"]


In [ ]:
#identify the feature importance
from sklearn.ensemble import RandomForestClassifier
from yellowbrick.model_selection import FeatureImportances

model = RandomForestClassifier(n_estimators=100)
viz = FeatureImportances(model)
viz.fit(X, y)
viz.show()

In [ ]:
# Create a dictionary that will map the feature name with its feature importance
feats = {}

# Loop through Feature
for feature, importance in zip(X.columns, model.feature_importances_):
    feats[feature] = importance # Add the name/value pair

# View our dictionary, but sorted in order of importance
sorted(feats.items(), key=lambda x: x[1], reverse=True)

In [ ]:
# We will pick top 50 features to be our important features

features = sorted(feats.items(), key=lambda x: x[1], reverse=True)[:50]
imp_features=[]
for f in features:
    imp_features.append(f[0])
#imp_features = tuple(imp_features)

In [17]:
LABEL_MAP = {
    -6: "Not Happy",
    -9: "Not Happy",
    1: "Not Happy",
    2: "Not Happy",
    3: "Not Happy",
    4: "Not Happy",
    5: "Not Happy",
    6: "Not Happy",
    7: "Happy",
    8: "Happy",
    9: "Happy",
    10: "Happy"
}

In [18]:
# Convert class labels into text
household_181["ratingnh"] = household_181["ratingnh"].map(LABEL_MAP)

In [20]:
household_181["ratingnh"]

0         Happy
1         Happy
2         Happy
3         Happy
4         Happy
          ...  
136240    Happy
136241    Happy
136242    Happy
136243    Happy
136244    Happy
Name: ratingnh, Length: 136245, dtype: object

In [19]:
from sklearn.preprocessing import LabelEncoder

# Extract our X and y data

y = household_181["ratingnh"]

# Encode our target variable
encoder = LabelEncoder().fit(y)
y = encoder.transform(y)
print(X.shape, y.shape)

(136245, 181) (136245,)


In [ ]:
from yellowbrick.features import RadViz

_ = RadViz(classes=encoder.classes_, alpha=0.35).fit_transform_show(X, y)

## Regression

In [ ]:
from sklearn import metrics
from sklearn.model_selection import KFold

from sklearn.linear_model import SGDClassifier
from sklearn import linear_model

In [ ]:
def get_internal_params(model):
    for attr in dir(model):
        if attr.endswith("_") and not attr.startswith("_"):
            print(attr, getattr(model, attr))

In [ ]:
#Perform SGD Classifier

model = SGDClassifier(loss="hinge", penalty="l2", max_iter=100)
model.fit(X,y)
get_internal_params(model)

In [ ]:
# Perform  Lasso Classifier Classification

model = linear_model.Ridge(alpha=1)
model.fit(X,y)
get_internal_params(model)